In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import shutil
import opendatasets as od
from sklearn.preprocessing import LabelEncoder
pd.set_option('mode.chained_assignment', None)
from tqdm import tqdm
import pysnooper

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
subdir = 'utils'
path = os.path.join(parent_dir, subdir)
sys.path.append(path)
from tqdm import tqdm

In [ ]:
# Get the data from here https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data
path_to_directory = os.path.abspath('../data/titanic')
if not os.path.exists(path_to_directory):
    # URL of the dataset
    dataset_url = 'https://www.kaggle.com/competitions/titanic/data'
    # Download the dataset to the specified location
    od.download(dataset_url)
    shutil.move(os.path.abspath('titanic'), path_to_directory)

train_data = 'train.csv'
test_data = 'test.csv'
train_df = pd.read_csv(os.path.join(path_to_directory, train_data))
test_df = pd.read_csv(os.path.join(path_to_directory, test_data))

In [ ]:
train_df = train_df.fillna(0)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use(['ipynb'])

In [ ]:
@pysnooper.snoop()
def create_something(iter):
    x = 0
    for i in range(iter):
        x = x+i**34

In [ ]:
create_something(10)

# Decision Treest

Recursive partitioning. Regularisation reduce the depth of the tree.

Advantages:
- white box models

Disadvantages:

In [ ]:
# Decision trees

class Decision_tree():
    def __init__(self, data, target, max_depth) -> None:
        self.data = data
        self.target = target
        self.seen = set()
        self.max_depth = max_depth
        
    def all_iteration(self):
        while self.max_depth >0:
            self.best_split_for_single_iter()
            print(self.seen)
            self.max_depth -= 1 
        return self.gini_values
    
    def best_split_for_single_iter(self):
        columns = [col for col in self.data.columns if col not in ['PassengerId', self.target, 'Name', 'Ticket'] ]
        self.gini_values = {}
        for col  in columns:
            for val in self.data[col].unique():
                if (col, str(val)) not in self.seen:
                    gini_weighted = self.split(col, val)
                    self.gini_values[col + '_split_' + str(val)] = gini_weighted
                else:
                    print('dont calculate gini on these', col, val)
                    
        # Find the minimum value of the dictionary and update the self.data
        min_key = min(self.gini_values, key=self.gini_values.get)
        col, val = min_key.split('_split_')
        print(col, val)
        self.seen.add((col, str(val)))
        return (col, val)
        
            
    def split(self, col, val):
        
        # Calculate probability for each node
        cls1 = self.data[self.data[col] == val]
        cls2 = self.data[self.data[col] != val]
        prob_list, n1, n2 = self.calculate_probability(cls1, cls2)
        
        # Calculate gini_index for each node
        g1, g2 = self.gini_index(prob_list)
        
        # Return weighted gini-impuritiny index
        return self.gini_impurity_for_split(g1, g2, n1, n2)
    
    def calculate_probability(self, cls1, cls2):
        ''''''
        prob_cls1 = []
        prob_cls2 = []
        n1 = len(cls1)
        n2 = len(cls2)
        for val in cls1[self.target].unique():
            temp_data = len(cls1[cls1[self.target] == val].copy(deep=True))
            prob_cls1.append(temp_data / n1)

        for val in cls2[self.target].unique():
            value2 = len(cls2[cls2[self.target] == val].copy(deep=True))
            prob_cls2.append(value2 / n2)
        return [prob_cls1, prob_cls2], n1, n2
        
    def gini_index(self, prob_list):
        g1 =  1 - np.sum([element **2 for element in prob_list[0]])
        g2 =  1 - np.sum([element **2 for element in prob_list[1]])
        return g1, g2
        
    def gini_impurity_for_split(self, g1, g2, n1, n2):
        return n1/(n1+n2) * g1 + n2/(n1+n2) * g2
        

In [15]:
decision_tree = Decision_tree(train_df, 'Survived', 5)
gini_dict = decision_tree.all_iteration()

KeyboardInterrupt: 

In [ ]:
# pd.options.plotting.backend = 'plotly'
plt.scatter(train_df['Sex'], train_df['Fare'])
# plt.scatter(train_df['Survived'], train_df['Sex'])

In [ ]:
train_df

In [ ]:
import seaborn as sns
cmap = sns.light_palette('red', as_cmap =  True)

train_df.style.background_gradient(cmap)